In [1]:
import pandas as pd
from math import log
import numpy as np

In [2]:
labels_dict = {
    "neutral" :    0,
    "happiness" :  1,
    "surprise" :   2,
    "sadness" :    3,
    "anger" :      4,
    "disgust" :    5,
    "fear" :       6, 
    "contempt":    7,
    "unknown":     8,
    "NF":          8
}

### Source: https://github.com/microsoft/FERPlus 

In [3]:
fer_new_path = "data/fer2013new.csv"

### Select emotion that received the best scores

In [4]:
fer_df = pd.read_csv(fer_new_path)
fer_df["max_emotion"] = fer_df.loc[:, 'neutral':'NF'].idxmax(axis=1)
fer_df.head()

,Usage,Image name,neutral,happiness,surprise,sadness,anger,disgust,fear,contempt,unknown,NF,max_emotion
0,Training,fer0000000.png,4,0,0,1,3,2,0,0,0,0,neutral
1,Training,fer0000001.png,6,0,1,1,0,0,0,0,2,0,neutral
2,Training,fer0000002.png,5,0,0,3,1,0,0,0,1,0,neutral
3,Training,fer0000003.png,4,0,0,4,1,0,0,0,1,0,neutral
4,Training,fer0000004.png,9,0,0,1,0,0,0,0,0,0,neutral


### Merge information from OpenFace 2.0 with Fer+

In [5]:
df = pd.read_csv("data/df_image_information.csv")
df.head()

,Unnamed: 0,face,confidence,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,gaze_1_z,gaze_angle_x,...,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,img_id
0,0,0,0.975,-0.164916,0.111523,-0.979982,-0.387946,0.182246,-0.903484,-0.286,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,34917
1,0,0,0.925,0.118285,0.234926,-0.964789,-0.226936,0.221684,-0.948344,-0.057,...,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,4650
2,0,0,0.925,0.185032,0.144629,-0.972032,-0.138360,0.130931,-0.981689,0.024,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,18947
3,0,0,0.875,0.162938,-0.260626,-0.951591,-0.103752,-0.255926,-0.961113,0.031,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,31507
4,0,0,0.925,0.151061,-0.125565,-0.980517,-0.112940,-0.239244,-0.964369,0.020,...,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,24652


In [6]:
fer_df["index"] = fer_df.index
result = pd.merge(df, fer_df, how='inner', right_on = 'index', left_on='img_id')
result.head()

,Unnamed: 0,face,confidence,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,gaze_1_z,gaze_angle_x,...,surprise,sadness,anger,disgust,fear,contempt,unknown,NF,max_emotion,index
0,0,0,0.975,-0.164916,0.111523,-0.979982,-0.387946,0.182246,-0.903484,-0.286,...,0,4,1,0,0,0,0,0,neutral,34917
1,0,0,0.925,0.118285,0.234926,-0.964789,-0.226936,0.221684,-0.948344,-0.057,...,2,2,0,1,0,0,0,0,neutral,4650
2,0,0,0.925,0.185032,0.144629,-0.972032,-0.138360,0.130931,-0.981689,0.024,...,1,0,7,0,1,0,0,0,anger,18947
3,0,0,0.875,0.162938,-0.260626,-0.951591,-0.103752,-0.255926,-0.961113,0.031,...,0,2,0,0,0,0,0,0,neutral,31507
4,0,0,0.925,0.151061,-0.125565,-0.980517,-0.112940,-0.239244,-0.964369,0.020,...,0,0,0,0,0,0,0,0,happiness,24652


### Clear data

In [7]:
result = result[np.isnan(result[" AU15_c"]) == False]
emotions = result.max_emotion
emotions_label = [labels_dict[x] for x in emotions]
result["emotion_label"] = emotions_label
result["emotion"] = emotions
result.head()

,Unnamed: 0,face,confidence,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,gaze_1_z,gaze_angle_x,...,anger,disgust,fear,contempt,unknown,NF,max_emotion,index,emotion_label,emotion
0,0,0,0.975,-0.164916,0.111523,-0.979982,-0.387946,0.182246,-0.903484,-0.286,...,1,0,0,0,0,0,neutral,34917,0,neutral
1,0,0,0.925,0.118285,0.234926,-0.964789,-0.226936,0.221684,-0.948344,-0.057,...,0,1,0,0,0,0,neutral,4650,0,neutral
2,0,0,0.925,0.185032,0.144629,-0.972032,-0.138360,0.130931,-0.981689,0.024,...,7,0,1,0,0,0,anger,18947,4,anger
3,0,0,0.875,0.162938,-0.260626,-0.951591,-0.103752,-0.255926,-0.961113,0.031,...,0,0,0,0,0,0,neutral,31507,0,neutral
4,0,0,0.925,0.151061,-0.125565,-0.980517,-0.112940,-0.239244,-0.964369,0.020,...,0,0,0,0,0,0,happiness,24652,1,happiness


### Create file with emotions only (!= "unknown", "NF")

In [8]:
result = result[np.isnan(result.emotion_label) == False]
result = result[result.emotion_label != 8] 
result.head()

,Unnamed: 0,face,confidence,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,gaze_1_z,gaze_angle_x,...,anger,disgust,fear,contempt,unknown,NF,max_emotion,index,emotion_label,emotion
0,0,0,0.975,-0.164916,0.111523,-0.979982,-0.387946,0.182246,-0.903484,-0.286,...,1,0,0,0,0,0,neutral,34917,0,neutral
1,0,0,0.925,0.118285,0.234926,-0.964789,-0.226936,0.221684,-0.948344,-0.057,...,0,1,0,0,0,0,neutral,4650,0,neutral
2,0,0,0.925,0.185032,0.144629,-0.972032,-0.138360,0.130931,-0.981689,0.024,...,7,0,1,0,0,0,anger,18947,4,anger
3,0,0,0.875,0.162938,-0.260626,-0.951591,-0.103752,-0.255926,-0.961113,0.031,...,0,0,0,0,0,0,neutral,31507,0,neutral
4,0,0,0.925,0.151061,-0.125565,-0.980517,-0.112940,-0.239244,-0.964369,0.020,...,0,0,0,0,0,0,happiness,24652,1,happiness


### Number of each emotion

In [9]:
annotation_dict = {
    "neutral" : 0,
    "happiness" : 0,
    "surprise" : 0,
    "sadness" : 0,
    "anger" : 0,
    "disgust" : 0,
    "fear" : 0,
    "contempt" : 0  
}

for i in result.index:
    try:
        annotation_dict[result.emotion[i]] = annotation_dict[result.emotion[i]]+1
    except:
        pass
    
annotation_dict

{'neutral': 10256,
 'happiness': 7375,
 'surprise': 3397,
 'sadness': 2786,
 'anger': 2225,
 'disgust': 192,
 'fear': 562,
 'contempt': 161}

### Write to file

In [10]:
result.to_csv("data/emotions.csv")

# Entropy

### Calculate entropies for each data sample in the set

In [11]:
entropys = []
for i in range(len(result)):
    try:
        entropy = 0
        if result.neutral[i]!= 0:
            entropy = entropy + log(result.neutral[i]/10, 2)*(result.neutral[i]/10)
        if result.happiness[i]!= 0:
            entropy = entropy + log(result.happiness[i]/10, 2)*(result.happiness[i]/10)
        if result.surprise[i]!= 0:
            entropy = entropy + log(result.surprise[i]/10, 2)*(result.surprise[i]/10)
        if result.sadness[i]!= 0:
            entropy = entropy + log(result.sadness[i]/10, 2)*(result.sadness[i]/10)
        if result.anger[i]!= 0:
            entropy = entropy + log(result.anger[i]/10, 2)*(result.anger[i]/10)
        if result.disgust[i]!= 0:
            entropy = entropy + log(result.disgust[i]/10, 2)*(result.disgust[i]/10)
        if result.fear[i]!= 0:
            entropy = entropy + log(result.fear[i]/10, 2)*(result.fear[i]/10)
        if result.contempt[i]!= 0:
            entropy = entropy + log(result.contempt[i]/10, 2)*(result.contempt[i]/10)
        entropys.append((-1)*entropy)
    except:
        entropys.append("Bad value")
    
result["entropy"] = entropys
result.head()

,Unnamed: 0,face,confidence,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,gaze_1_z,gaze_angle_x,...,disgust,fear,contempt,unknown,NF,max_emotion,index,emotion_label,emotion,entropy
0,0,0,0.975,-0.164916,0.111523,-0.979982,-0.387946,0.182246,-0.903484,-0.286,...,0,0,0,0,0,neutral,34917,0,neutral,1.36096
1,0,0,0.925,0.118285,0.234926,-0.964789,-0.226936,0.221684,-0.948344,-0.057,...,1,0,0,0,0,neutral,4650,0,neutral,1.76096
2,0,0,0.925,0.185032,0.144629,-0.972032,-0.138360,0.130931,-0.981689,0.024,...,0,1,0,0,0,anger,18947,4,anger,1.35678
3,0,0,0.875,0.162938,-0.260626,-0.951591,-0.103752,-0.255926,-0.961113,0.031,...,0,0,0,0,0,neutral,31507,0,neutral,0.721928
4,0,0,0.925,0.151061,-0.125565,-0.980517,-0.112940,-0.239244,-0.964369,0.020,...,0,0,0,0,0,happiness,24652,1,happiness,-0


### Remove "Bad values" from dataset and samples with an entropy greater or equal than 1

In [12]:
result = result[result.entropy != "Bad value"]
result = result[result.entropy < 1.]
result.head()

,Unnamed: 0,face,confidence,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,gaze_1_z,gaze_angle_x,...,disgust,fear,contempt,unknown,NF,max_emotion,index,emotion_label,emotion,entropy
3,0,0,0.875,0.162938,-0.260626,-0.951591,-0.103752,-0.255926,-0.961113,0.031,...,0,0,0,0,0,neutral,31507,0,neutral,0.721928
4,0,0,0.925,0.151061,-0.125565,-0.980517,-0.112940,-0.239244,-0.964369,0.020,...,0,0,0,0,0,happiness,24652,1,happiness,-0
5,0,0,0.875,0.209148,0.413029,-0.886377,-0.190284,0.403735,-0.894869,0.011,...,0,0,0,1,0,happiness,28912,1,happiness,0.136803
8,0,0,0.925,-0.037131,0.124201,-0.991562,-0.324617,0.086311,-0.941899,-0.185,...,0,0,0,1,0,happiness,28945,1,happiness,0.589735
9,0,0,0.975,0.171331,0.160390,-0.972070,-0.151369,0.174661,-0.972924,0.010,...,0,0,1,0,0,anger,3554,4,anger,0.468996


### Write to file

In [13]:
result.to_csv("data/emotions_with_entropy.csv")

### Number of each emotion

In [14]:
annotation_dict = {
    "neutral" : 0,
    "happiness" : 0,
    "surprise" : 0,
    "sadness" : 0,
    "anger" : 0,
    "disgust" : 0,
    "fear" : 0,
    "contempt" : 0  
}

for i in range(len(result)):
    try:
        annotation_dict[result.emotion[i]] = annotation_dict[result.emotion[i]]+1
    except:
        pass
    
annotation_dict

{'neutral': 4393,
 'happiness': 3069,
 'surprise': 1379,
 'sadness': 1149,
 'anger': 931,
 'disgust': 72,
 'fear': 227,
 'contempt': 61}